In [ ]:
import numpy as np
from keras.callbacks import EarlyStopping
from keras.models import Sequential,load_model
from keras.layers import Dense,Dropout, Conv2D, MaxPooling2D,Flatten,Softmax,Activation
from keras.optimizers import SGD, Adam, RMSprop,Nadam, Adadelta
#from keras.utils import np_utils
import scipy.misc
import glob, random
import os
from keras import backend as K


%matplotlib inline 
from skimage.io import imshow, imread, imsave 
os.chdir("C:/Users/ПОЛЬЗОВАТЕЛЬ/Desktop")

In [ ]:
Batches = 20
num_epochs = 1
N=40

In [ ]:
def Val():
    n=0
    path = glob.glob("Выборка-Val/(*) *.jpg")
    print(len(path))
    valid_in=[]
    valid_out=[]
    for i in path:
        imge= imread(i)
        tabe = np.array(imge)/255
        valid_in.append(tabe)
        if int(i[-5]) == 1:
            valid_out.append((1,0))
        else:
            valid_out.append((0,1))
        n+=1  
    return [valid_in,valid_out]
        
def reform(in_):
    return np.array(in_)

        
def shuffles():
        n=0
        path = glob.glob("Выборка\(*) 1.jpg")
        print("Один - ",len(path))
        random.shuffle(path)
        out=[]
        out.append(path[:450])

        path = glob.glob("Выборка\(*) 0.jpg")
        print("Много - ",len(path))
        random.shuffle(path)
        out.append(path[:450])
        return out      
        
def load(path):
    test_in = []
    test_out = []
    for i in path[0]:
        test_out.append((1,0))
        img = imread(i)
        tabe = np.array(img)/255
        test_in.append(tabe)
    for i in path[1]:
        test_out.append((0,1))
        img = imread(i)
        tabe = np.array(img)/255
        test_in.append(tabe)    
    
    return ([test_in,test_out])
        
def sborka():
    path=shuffles()
    Load = load(path)
    Load1 = np.zeros((len(Load[0]),400,400,3))
    Load2 = []
    for i in range(len(Load[0])):
        Load1[i] = reform(Load[0][0])
        Load2.append(Load[1][0])
        del(Load[0][0],Load[1][0])
    Load2 = np.array(Load2)
    return ([Load1,Load2])

def Load_Net(path):
    Net = load_model(path)
    Net.compile(loss = "logcosh",optimizer=Adadelta(),metrics=['accuracy'])
    return(Net)
    
    

In [ ]:
Valid = Val()

In [ ]:
Valid1 = np.zeros((len(Valid[0]),400,400,3))
for i in range(len(Valid[0])):
    Valid1[i]=reform(Valid[0][i])
    Valid[0][i]=0
Valid = np.array(Valid[1])


In [ ]:
i =9
imshow(Valid1[i])
print(Valid[i])

In [ ]:
print("Сеть получит {} выборок обучения ".format(N))
Min_Loss = float(input("Пороговое значение ошибки - "))
for i in range(N):    
    print("Выборка № ",i+1)
    Load = sborka()
    Net = Load_Net("Од.мн.h5") 
    Net.fit(Load[0], 
            Load[1],
            batch_size = Batches, 
            epochs = num_epochs,
            validation_data = (Valid1,Valid)
           )
    accuracy = Net.evaluate(Valid1,Valid)[0]
    if accuracy < Min_Loss:
            print("Ошибка валидационной выборки - {}, что меньше чем наилучший показатель - {}".format(accuracy,Min_Loss))
            Min_Loss=accuracy
            Net.save('Од.мн_loss({}).h5'.format(round(accuracy,4)))
    Net.save('Од.мн.h5')
    del (Load,Net)#,accuracy)

In [ ]:
input_shape = (400,400,3)
Net = Sequential()
Net.add(Conv2D(5,input_shape = input_shape,kernel_size = (3,3),activation = K.relu))
Net.add(MaxPooling2D(pool_size = (3,3)))
Net.add(Conv2D(10,kernel_size = (3,3),activation = K.relu))
Net.add(MaxPooling2D(pool_size = (3,3)))
Net.add(Conv2D(15,kernel_size = (3,3),activation = K.relu))
Net.add(MaxPooling2D(pool_size = (3,3)))
Net.add(Conv2D(30,kernel_size = (3,3),activation = K.relu))
Net.add(MaxPooling2D(pool_size = (3,3)))
Net.add(Dropout(0.5))
Net.add(Flatten())
Net.add(Dense(50,activation = K.relu))
Net.add(Dropout(0.5))
Net.add(Dense(50,activation = K.relu))
Net.add(Dropout(0.5))
Net.add(Dense(50,activation = K.relu))
Net.add(Dropout(0.5))
Net.add(Dense(2,activation = K.softmax))
Net.save('Од.мн.h5')
Net.summary()
del(Net)